In [1]:
import random
import numpy as np
import pandas as pd
from sklearn.datasets import make_spd_matrix
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from numpy.random import normal, multivariate_normal
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from dgp import DGP
from inference import Inference

In [2]:
def plot_hist(tau1, tau0):
    
    fig, (ax_1, ax_2) = plt.subplots(1, 2, figsize=(14, 5))
    ax_1.hist(tau1, color="b")
    ax_1.set_xlabel("tau1")
    ax_1.set_ylabel("Count")
    ax_1.axvline(x=np.mean(tau1), ymin=0, ymax=3000, color="k", linestyle="--")

    ax_2.hist(tau0, color="r")
    ax_2.set_xlabel("tau0")
    ax_2.set_ylabel("Count")
    ax_2.axvline(x=np.mean(tau0), ymin=0, ymax=3000, color="k", linestyle="--")
    return fig

def reject_prob(n, modelY='1', modelDA='1', ate=0, ntrials=1000):
    phi_tau1s, phi_tau0s, phi_thetas = np.zeros(ntrials), np.zeros(ntrials), np.zeros(ntrials)
    for i in range(ntrials):
        dgp = DGP(modelY,modelDA,n,tau=ate)
        inf = Inference(dgp.Y, dgp.D, dgp.A, modelDA, tuple_idx=dgp.tuple_idx, tau=dgp.tau)
        phi_tau1, phi_tau0, phi_theta = inf.inference()
        phi_tau1s[i] = phi_tau1
        phi_tau0s[i] = phi_tau0
        phi_thetas[i] = phi_theta
    return phi_tau1s, phi_tau0s, phi_thetas

def risk(n, modelY='1', modelDA='1', ate=0, ntrials=1000):
    tau1s, tau0s, thetas = np.zeros(ntrials), np.zeros(ntrials), np.zeros(ntrials)
    for i in range(ntrials):
        dgp = DGP(modelY,modelDA,n,tau=ate)
        inf = Inference(dgp.Y, dgp.D, dgp.A, modelDA, tuple_idx=dgp.tuple_idx, tau=dgp.tau)
        tau1s[i] = inf.tau1
        tau0s[i] = inf.tau0
        thetas[i] = inf.theta
    return tau1s, tau0s, thetas

In [3]:
# report MSE
for i in range(6):
    print("ModelY={}".format(i+1))
    mse = np.zeros((11,3))
    for j in range(11):
        tau1s, tau0s, thetas = risk(1000, modelY=str(i+1), modelDA=str(j+1), ntrials=2000)
        mse[j,0] = np.mean(thetas**2)
        mse[j,1] = np.mean(tau1s**2)
        mse[j,2] = np.mean(tau0s**2)
    mse = mse/mse[7]
    mse = mse.T
    for r in range(3):
        for k in range(11):
            if k<10:
                print("{:.3f} & ".format(mse[r,k]), end = '')
            else:
                print("{:.3f} \\\\".format(mse[r,k]))

ModelY=1
8.989 & 8.759 & 0.905 & 0.915 & 9.357 & 8.999 & 0.969 & 1.000 & 2.909 & 1.431 & 1.254 \\
8.628 & 8.851 & 4.882 & 4.835 & 9.141 & 8.958 & 9.314 & 1.000 & 2.723 & 1.387 & 1.185 \\
9.543 & 9.518 & 5.160 & 5.088 & 9.794 & 9.678 & 10.063 & 1.000 & 3.094 & 1.527 & 1.321 \\
ModelY=2
9.305 & 9.460 & 9.369 & 9.202 & 9.781 & 9.773 & 17.279 & 1.000 & 2.992 & 1.531 & 1.302 \\
9.450 & 9.650 & 5.327 & 5.200 & 9.623 & 9.374 & 9.432 & 1.000 & 3.054 & 1.538 & 1.370 \\
9.503 & 9.091 & 5.155 & 5.055 & 9.705 & 9.694 & 9.007 & 1.000 & 3.033 & 1.485 & 1.239 \\
ModelY=3
8.519 & 8.659 & 8.838 & 8.785 & 8.709 & 8.713 & 15.393 & 1.000 & 2.895 & 1.447 & 1.264 \\
8.877 & 8.478 & 5.118 & 5.012 & 8.480 & 9.341 & 8.313 & 1.000 & 2.975 & 1.414 & 1.323 \\
8.753 & 8.927 & 4.783 & 4.866 & 9.242 & 8.655 & 8.167 & 1.000 & 2.992 & 1.398 & 1.183 \\
ModelY=4
3.633 & 3.753 & 3.223 & 3.220 & 3.826 & 4.185 & 5.794 & 1.000 & 1.760 & 1.212 & 1.091 \\
9.267 & 9.126 & 5.417 & 5.296 & 8.993 & 9.470 & 9.467 & 1.000 & 2.998 &

In [4]:
# report Reject Probability with ate=0
for i in range(6):
    print("ModelY={}".format(i+1))
    prob = np.zeros((6,3))
    for j, d in enumerate(['1', '2', '8', '8p', '9', '10']):
        phi_tau1s, phi_tau0s, phi_theta = reject_prob(1000, modelY=str(i+1), modelDA=d, ntrials=2000)
        prob[j,0] = np.mean(phi_theta)
        prob[j,1] = np.mean(phi_tau1s)
        prob[j,2] = np.mean(phi_tau0s)
    prob = prob.T
    for r in range(3):
        for k in range(6):
            if k<5:
                print("{:.3f} & ".format(prob[r,k]), end = '')
            else:
                print("{:.3f} \\\\".format(prob[r,k]))

ModelY=1
0.043 & 0.051 & 0.059 & 0.059 & 0.051 & 0.049 \\
0.046 & 0.052 & 0.047 & 0.051 & 0.058 & 0.056 \\
0.050 & 0.046 & 0.054 & 0.052 & 0.050 & 0.048 \\
ModelY=2
0.053 & 0.049 & 0.054 & 0.043 & 0.048 & 0.057 \\
0.057 & 0.054 & 0.046 & 0.051 & 0.049 & 0.059 \\
0.050 & 0.050 & 0.043 & 0.052 & 0.049 & 0.054 \\
ModelY=3
0.049 & 0.051 & 0.054 & 0.045 & 0.044 & 0.052 \\
0.047 & 0.052 & 0.051 & 0.051 & 0.051 & 0.053 \\
0.053 & 0.048 & 0.049 & 0.054 & 0.050 & 0.050 \\
ModelY=4
0.057 & 0.043 & 0.048 & 0.055 & 0.056 & 0.046 \\
0.053 & 0.050 & 0.050 & 0.048 & 0.051 & 0.044 \\
0.048 & 0.045 & 0.051 & 0.050 & 0.048 & 0.044 \\
ModelY=5
0.052 & 0.051 & 0.052 & 0.051 & 0.048 & 0.049 \\
0.058 & 0.053 & 0.048 & 0.043 & 0.060 & 0.059 \\
0.051 & 0.057 & 0.043 & 0.048 & 0.051 & 0.059 \\
ModelY=6
0.058 & 0.058 & 0.048 & 0.048 & 0.045 & 0.054 \\
0.054 & 0.045 & 0.051 & 0.048 & 0.041 & 0.051 \\
0.046 & 0.053 & 0.049 & 0.051 & 0.039 & 0.060 \\


In [5]:
for i in range(6):
    print("ModelY={}".format(i+1))
    prob = np.zeros((6,3))
    for j, d in enumerate(['1', '2', '8', '8p', '9', '10']):
        phi_tau1s, phi_tau0s, phi_theta = reject_prob(1000, modelY=str(i+1), modelDA=d, ate=0.02, ntrials=2000)
        prob[j,0] = np.mean(phi_theta)
        prob[j,1] = np.mean(phi_tau1s)
        prob[j,2] = np.mean(phi_tau0s)
    prob = prob.T
    for r in range(3):
        for k in range(6):
            if k<5:
                print("{:.3f} & ".format(prob[r,k]), end = '')
            else:
                print("{:.3f} \\\\".format(prob[r,k]))

ModelY=1
0.192 & 0.178 & 0.881 & 0.866 & 0.435 & 0.741 \\
0.111 & 0.115 & 0.602 & 0.590 & 0.232 & 0.444 \\
0.117 & 0.116 & 0.622 & 0.603 & 0.244 & 0.452 \\
ModelY=2
0.188 & 0.181 & 0.891 & 0.885 & 0.444 & 0.760 \\
0.123 & 0.111 & 0.614 & 0.612 & 0.254 & 0.466 \\
0.124 & 0.116 & 0.610 & 0.605 & 0.248 & 0.464 \\
ModelY=3
0.208 & 0.193 & 0.886 & 0.878 & 0.438 & 0.751 \\
0.117 & 0.130 & 0.606 & 0.593 & 0.258 & 0.451 \\
0.137 & 0.126 & 0.606 & 0.590 & 0.255 & 0.473 \\
ModelY=4
0.137 & 0.131 & 0.423 & 0.424 & 0.246 & 0.350 \\
0.081 & 0.072 & 0.345 & 0.332 & 0.129 & 0.233 \\
0.167 & 0.162 & 0.332 & 0.329 & 0.254 & 0.287 \\
ModelY=5
0.051 & 0.050 & 0.052 & 0.052 & 0.051 & 0.059 \\
0.050 & 0.050 & 0.055 & 0.049 & 0.046 & 0.052 \\
0.050 & 0.045 & 0.048 & 0.049 & 0.048 & 0.050 \\
ModelY=6
0.191 & 0.175 & 0.880 & 0.884 & 0.442 & 0.720 \\
0.110 & 0.108 & 0.612 & 0.624 & 0.258 & 0.435 \\
0.120 & 0.121 & 0.601 & 0.594 & 0.248 & 0.452 \\
